In [1]:
import datajoint as dj
import numpy as np
import pymeshfix

In [2]:
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [3]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [4]:
segment_id = 648518346349504955
segment_id = 648518346341366885
new_dict=dict(segmentation=2,segment_id=segment_id)
vertices_ar,triangles_ar = (ta3p100.Decimation35 & new_dict).fetch("vertices","triangles")

vertices = vertices_ar[0]
triangles = triangles_ar[0]
print(triangles.shape)

(673206, 3)


In [5]:
#try passing the mesh through the meshfix and see if will work then
#PYMESH METHOD FOR GENERATING THE WATER-TIGHT/NON-MANIFOLD MESH
pymesh_type = "_pymesh_vox_default"
#joincomp_setting = False
#remove_smallest_components_setting = False


import time
start_time = time.time()

#import the mesh from the off file instead

meshfix = pymeshfix.MeshFix(vertices,triangles)
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
#vertices_new = meshfix.v
#faces_new = meshfix.f
#write_Whole_Neuron_Off_file(str(segment_id) + pymesh_type,vertices_new,faces_new)
print(f"Pymeshfix {pymesh_type} shrinkwrap took {time.time()-start_time} seconds")


Pymeshfix _pymesh_vox_default shrinkwrap took 311.4419279098511 seconds


In [6]:
vertices_new = meshfix.v
faces_new = meshfix.f
write_Whole_Neuron_Off_file(str(segment_id) + "pymesh_17_attach",vertices_new,faces_new)

Done writing OFF file


('/notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346341366885pymesh_17_attach',
 'neuron_648518346341366885pymesh_17_attach',
 '/notebooks/Pymeshfix_vs_Voxelization/temp')

In [7]:
dir(pymeshfix.MeshFix)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'load_arrays',
 'mesh',
 'plot',
 'repair',
 'write']